Video url : https://www.youtube.com/watch?v=2Hm8eEHsgls&t=50s
- Runs before main and startup file calls main
- Some part of the startup code is processor dependent
- Startup code takes care of vector table placement in code memory as required by the arm cortex processor
- Also takes care of stack initialization
- Repsponsible for .data, .bss section initialization in main memory. It does this by copying these two sections from flash to sram and then calls main before that main should not be called. 

## Basic functions of startup code
1. Create a vector table 
2. Initialize .bss and .data section in sram
3. Call main

This is what we will be writing
![image](images/flashMemoryOrganization.png)

## What is a vector table??

- **Definition**:  
  A vector table is a data structure used by a microcontroller to manage interrupt and exception handling. It contains addresses (pointers) to the corresponding handler functions.

- **Purpose**:  
  It directs the processor to the correct Interrupt Service Routine (ISR) or exception handler when an interrupt or exception occurs.

- **Location**:  
  Typically located at a fixed address in memory (e.g., `0x00000000` for ARM Cortex-M devices by default).

- **Structure**:
  - The first entry often contains the **initial stack pointer**.
  - Subsequent entries contain addresses of **ISR or exception handlers**.
  - Each entry corresponds to a specific interrupt or exception vector (e.g., Reset, NMI, HardFault).

- **Usage**:
  - On an interrupt, the processor uses the interrupt number to index into the vector table.
  - It fetches the address from the table and jumps to that handler function.

- **Customization**:
  - Developers can modify the vector table to point to their own custom ISR implementations.
  - Some systems allow relocating the vector table to a different memory region (e.g., using the `VTOR` register in ARM Cortex-M).

- **Example (Simplified)**:
  ```c
  const void* vector_table[] __attribute__((section(".vectors"))) = {
      (void*) &stack_top,       // Initial Stack Pointer
      Reset_Handler,            // Reset Handler
      NMI_Handler,              // NMI Handler
      HardFault_Handler,        // HardFault Handler
      // ... other handlers
  };

Lets sya the vector table has 96 entries and each entry takes four bytes. We need to assign 388 bytes of total memory for the vector table. The way we know this by looking at the positions in the vector table. 

We explicitly mention that we are going to define a separate section that stores the vector information and not aligning with .data segment

### How to tell the compiler to move the data to a different section?
We can use the compiler attribute
You can search for this using the variable attribute in the [GCC GNU Documentation: Variable attributes](https://gcc.gnu.org/onlinedocs/gcc/Common-Variable-Attributes.html). And there, you will find the `section` attribute. 


> Normally, the compiler places the objects it generates in sections like data and bss. Sometimes, however, you need additional sections, or you need certain particular variables to appear in special sections, for example to map to special hardware. The section attribute specifies that a variable (or function) lives in a particular section. 

Example usage to force a structure to a desired section is `struct duart a __attribute__ ((section ("DUART_A"))) = { 0 };`

In our case, the section name is `.isr_vector`. Actually chatgpt clarified that this name really does not need to be .isr_vector. Here, the instructor named it as such. But in reality, you can almost name it anything. The only thing to remember is that this exact name should be rewritten later in the linker script. 


Once we are done compiling the startup file after adjusting the make file. We can objdump to see the new section

![image](images/startupFileSections.png)


notice the new `.isr_vector` section that appeared. 

We can also do the same `__attribute__ (section_xxx)` for various functions as well.

### Weak and Alias attributes
As you have already seen there anre many function handlers in place in the vector table. And also,you only use a couple of interrupts in your project anyway. Lets call this a `default_handler`. 


> Weak attributes let programmer to override already defined weak function (dummy) with the same function name

> Alias attributes lets programmer to give alias name to a function

`void NMI_Handler(void) __attribute__((alias("Default_Handler")))`

Here,`Default_handler` is an alias function name for `NMI_Handler`.

Now lets go ahead and make this weak as well. `void NMI_Handler(void) __attribute__((weak, alias("Default_Handler")))`

> Now we are making this `weak`. This is because, we are allowing programmer to override this function with same function name in main application. There, programmer can implement real implementation of handling that expection. 

So, in your original application file like `main.c`, you just declare

```
This is main.c
void NMI_Handler(void)
{
    // Your NMI handler implementation
}
```

So, now, when an NMI actually occurs, the control will not go to the default handler as it is declared as weak. It instead jumps to the NMI handler code you wrote in your `main.c` file.

## Writing startup code to initialize .data and .bss sections in SRAM

So far we have created the vector table for the microcontroller. Now, we will work towards initializaing the .data nad .bss sections in the SRAM. This will be written in the `reset_handler`. The basic structure is

1. Copy .data section to SRAM
2. Initialize the .bss section to zero in SRAM
3. Call the initialzation function of the standard library (optional)
4. Call the `main()` function

## How to do data copy?

![image](images/dataCopy.png)

As shown in the image above, we need to know where the data is stored in the flash memory to begin copying the data from the Flash to the SRAM. But for that we need to know the boundaries of the data sections in Flash memory. Therefore, the boundary information is supposed to be imported from the linker script to the startup file. The information that we care about is, `_edata`, `_sdata` and `_etext`.



### Completing the startup file
Please read this after reading up linkerscript.ipymb

By now, we have the boundary datas for the .data, .bss sections that need to be moved to the SRAM. 

> The good thing is theat the symbols inside the linker script are avaibale nside the C source files using the extern keyword. To know what symbols exist inside the data, we can use the symbol table. To see the symbol table for a project, 
`arm-none-eabi-nm final.elf`

Note that there is no value to the symbol. We must get the address. So, we compute the size of the data segment in our `startup.c` file as 
`uint32_t size = &_edata - &_sdata;`

So, now we can copy data section from flash to SRAM. Note that the start of data section is the end of text (symbol: `_etext`)
```
  // copy .data section to SRAM
  uint32_t size = (uint32_t)&_edata - (uint32_t)&_sdata;
  uint8_t *pDst = (uint8_t *)&_sdata; // SRAM
  uint8_t *pSrc = (uint8_t *)&_etext; // Flash

  for (uint32_t i = 0; i < size; i++) {
    *pDst++ = *pSrc++;
  }

```

Similar to that, we can copy the bss section. But this time, we zero out the data in order to prepre the bss section
  ```
  size = (uint32_t)&_ebss - (uint32_t)&_sbss;
  pDst = (uint8_t *)&_sbss;  // SRAM
  pSrc = (uint8_t *)&_edata; // Flash

  for (uint32_t i = 0; i < size; i++) {
    *pDst++ = 0;
  }
  ```

And after all that, we call the main function.
  